In [1]:
!pip install ipywidgets
!pip install scikit-multilearn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 3.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.model_selection import train_test_split
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import matplotlib.pyplot as plt
import seaborn as sns
import os


In [3]:
resume_templates = {
    "Data Science": [
        "Proficient in Python, Pandas, and Machine Learning",
        "Experience in data visualization with Matplotlib and Seaborn",
        "Built predictive models using scikit-learn and TensorFlow"
    ],
    "Software Development": [
        "Developed web applications using Django and React",
        "Strong foundation in object-oriented programming",
        "Familiar with REST APIs and Git version control"
    ],
    "Human Resources": [
        "Handled end-to-end recruitment and onboarding",
        "Managed employee relations and compliance",
        "Experienced in payroll processing and HRMS tools"
    ],
    "UI/UX Design": [
        "Designed mobile interfaces using Figma and Adobe XD",
        "Conducted user research and usability testing",
        "Skilled in creating wireframes and prototypes"
    ]
}

def generate_resume_dataset(n_samples=300):
    data = []
    roles = list(resume_templates.keys())

    for _ in range(n_samples):
        selected = np.random.choice(roles, size=np.random.randint(1, 3), replace=False)
        parts = [np.random.choice(resume_templates[role]) for role in selected]
        resume_text = " ".join(parts)
        labels = [1 if role in selected else 0 for role in roles]

        data.append({
            "Resume": resume_text,
            "Data Science": labels[0],
            "Software Development": labels[1],
            "Human Resources": labels[2],
            "UI/UX Design": labels[3]
        })

    return pd.DataFrame(data)

# Load or generate dataset
if not os.path.exists("resume_data.csv"):
    df = generate_resume_dataset()
    df.to_csv("resume_data.csv", index=False)
else:
    df = pd.read_csv("resume_data.csv")

df.head()


,Resume,Data Science,Software Development,Human Resources,UI/UX Design
0,Managed employee relations and compliance,0,0,1,0
1,Built predictive models using scikit-learn and...,1,0,0,1
2,Built predictive models using scikit-learn and...,1,0,0,0
3,Experience in data visualization with Matplotl...,1,0,0,0
4,Managed employee relations and compliance,0,0,1,0


In [4]:
X = df["Resume"]
y = df[["Data Science", "Software Development", "Human Resources", "UI/UX Design"]]

# Vectorization
vectorizer = TfidfVectorizer(stop_words="english", max_features=2000)
X_vec = vectorizer.fit_transform(X)

# Binary Relevance + Naive Bayes
classifier = BinaryRelevance(MultinomialNB())
classifier.fit(X_vec, y)

print("✅ Resume classifier model trained!")


✅ Resume classifier model trained!


In [5]:
resume_input = widgets.Textarea(
    value='',
    placeholder='Paste resume content here...',
    description='Resume:',
    layout=widgets.Layout(width='90%', height='150px')
)

threshold_slider = widgets.FloatSlider(
    value=0.4,
    min=0,
    max=1,
    step=0.05,
    description='Threshold:',
    continuous_update=False
)

submit_button = widgets.Button(
    description='Classify Resume',
    button_style='success'
)

output = widgets.Output()


In [6]:
def classify_resume(b):
    with output:
        clear_output()
        resume = resume_input.value
        if not resume.strip():
            print("⚠️ Please paste some resume content.")
            return

        vec = vectorizer.transform([resume])
        probs = classifier.predict_proba(vec).toarray()[0]
        threshold = threshold_slider.value

        categories = list(resume_templates.keys())
        predicted = [cat for cat, p in zip(categories, probs) if p >= threshold]

        display(HTML(f"""
        <h3>📄 Resume Content:</h3>
        <p>{resume}</p>
        <h3>🧭 Predicted Job Categories (Threshold = {threshold}):</h3>
        <b>{", ".join(predicted) if predicted else "No matching category found"}</b>
        """))

        display(HTML("<h4>📊 Confidence Scores:</h4>"))
        for cat, score in zip(categories, probs):
            display(HTML(f"{cat}: <b>{score*100:.1f}%</b>"))

        # Plot
        plt.figure(figsize=(8, 4))
        bars = plt.bar(categories, probs*100, color=['#4C72B0', '#55A868', '#C44E52', '#8172B3'])
        plt.axhline(y=threshold*100, color='red', linestyle='--')
        plt.title("Resume Classification Confidence")
        plt.ylabel("Confidence (%)")
        plt.ylim(0, 100)
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                     f'{height:.1f}%', ha='center', va='bottom')
        plt.show()

submit_button.on_click(classify_resume)


In [7]:
display(HTML("<h2>🤖 Resume Screening Assistant (with Classifier)</h2>"))
display(widgets.VBox([
    resume_input,
    threshold_slider,
    submit_button,
    output
]))
